Notebook to visualize results

input:
- tables/median*.csv
output:
- visualizations
- results

In [1]:
import os
import itertools

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pingouin as pg
import scikit_posthocs as sp
import seaborn as sns
#from cliffs_delta import cliffs_delta
from scipy import stats



In [2]:
cpuep_tables_dir = "D:/GAISSA/energy-repo/repo_sept_last_update/cpuep_nov_results/nov_cpuep_00/tables_nov/"
cudaep_tables_dir = "D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/nov_cudaep_00/tables_nov/"



In [35]:
variables =['global_energy',
                "avg_cpu_usage_config",'avg_used_memory_pct_config',
                'avg_utilization_gpu_config','avg_utilization_memory_config','avg_used_memory_pct_mib',
                'avg_Load', 'duration'
                ]
dependent_variable = variables[-1]
print(f'using variable: {dependent_variable}')

using variable: duration


In [36]:
def remove_condition_rows(condition_to_remove,df):
    # example: condition = merged_df['label']  == 'idle_idle'
    df1 = df.copy()
    return df1[~condition_to_remove]

In [37]:
# if dependent_variable in ['global_energy',]:
#     cpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_cpu/tables/median_energy_values.csv", index_col=None, header=0)
#     gpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_gpu_test/tables/median_energy_values.csv", index_col=None, header=0)
# elif dependent_variable in ["avg_cpu_usage_config",'avg_used_memory_pct_config',]:
#     cpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_cpu/tables/median_resource_energi.csv", index_col=None, header=0)
#     gpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_gpu_test/tables/median_resource_energi.csv", index_col=None, header=0)
# elif dependent_variable in ['avg_utilization_gpu_config','avg_utilization_memory_config','avg_used_memory_pct_mib',]:
#     cpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_cpu/tables/median_resource_nvidia.csv", index_col=None, header=0)
#     gpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_gpu_test/tables/median_resource_nvidia.csv", index_col=None, header=0)
# elif dependent_variable in ['avg_Load',] :
#     cpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_cpu/tables/median_resource_wattmeter.csv", index_col=None, header=0)
#     gpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_gpu_test//tables/median_resource_wattmeter.csv", index_col=None, header=0)
# elif dependent_variable in ['duration', ]:
#     cpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_cpu/tables/median_time_inference.csv", index_col=None, header=0)
#     gpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_gpu_test/tables/median_time_inference.csv", index_col=None, header=0)

if dependent_variable in ['global_energy',]:
    cpu_ep_data = pd.read_csv(f"{cpuep_tables_dir}median_energy_values.csv", index_col=None, header=0)
    gpu_ep_data = pd.read_csv(f"{cudaep_tables_dir}median_energy_values.csv", index_col=None, header=0)
elif dependent_variable in ["avg_cpu_usage_config",'avg_used_memory_pct_config',]:
    cpu_ep_data = pd.read_csv(f"{cpuep_tables_dir}median_resource_energi.csv", index_col=None, header=0)
    gpu_ep_data = pd.read_csv(f"{cudaep_tables_dir}median_resource_energi.csv", index_col=None, header=0)
elif dependent_variable in ['avg_utilization_gpu_config','avg_utilization_memory_config','avg_used_memory_pct_mib',]:
    cpu_ep_data = pd.read_csv(f"{cpuep_tables_dir}median_resource_nvidia.csv", index_col=None, header=0)
    gpu_ep_data = pd.read_csv(f"{cudaep_tables_dir}median_resource_nvidia.csv", index_col=None, header=0)
elif dependent_variable in ['avg_Load',] :
    cpu_ep_data = pd.read_csv(f"{cpuep_tables_dir}median_resource_wattmeter.csv", index_col=None, header=0)
    gpu_ep_data = pd.read_csv(f"{cudaep_tables_dir}median_resource_wattmeter.csv", index_col=None, header=0)
elif dependent_variable in ['duration', ]:
    cpu_ep_data = pd.read_csv(f"{cpuep_tables_dir}median_time_inference.csv", index_col=None, header=0)
    gpu_ep_data = pd.read_csv(f"{cudaep_tables_dir}median_time_inference.csv", index_col=None, header=0)

print(len(cpu_ep_data))
print(len(gpu_ep_data))

48
36


In [38]:
print(len(cpu_ep_data['label'].unique()))
print(cpu_ep_data['label'].unique())

print(len(gpu_ep_data['label'].unique()))
print(gpu_ep_data['label'].unique())


48
['onnx_bloomz-1b1' 'onnx_bloomz-560m' 'onnx_codegemma-2b'
 'onnx_codeparrot-small' 'onnx_phi2' 'onnx_pythia-410m' 'onnx_pythia1-4b'
 'onnx_stablecode-3b' 'onnx_stablecode-3b-completion'
 'onnx_starcoderbase-1b' 'onnx_tiny-starcoder' 'onnx_tinyllama'
 'ov_bloomz-1b1' 'ov_bloomz-560m' 'ov_codegemma-2b' 'ov_codeparrot-small'
 'ov_phi2' 'ov_pythia-410m' 'ov_pythia1-4b' 'ov_stablecode-3b'
 'ov_stablecode-3b-completion' 'ov_starcoderbase-1b' 'ov_tiny-starcoder'
 'ov_tinyllama' 'torch_bloomz-1b1' 'torch_bloomz-560m'
 'torch_codegemma-2b' 'torch_codeparrot-small' 'torch_phi2'
 'torch_pythia-410m' 'torch_pythia1-4b' 'torch_stablecode-3b'
 'torch_stablecode-3b-completion' 'torch_starcoderbase-1b'
 'torch_tiny-starcoder' 'torch_tinyllama' 'torchscript_bloomz-1b1'
 'torchscript_bloomz-560m' 'torchscript_codegemma-2b'
 'torchscript_codeparrot-small' 'torchscript_phi2'
 'torchscript_pythia-410m' 'torchscript_pythia1-4b'
 'torchscript_stablecode-3b' 'torchscript_stablecode-3b-completion'
 'torchsc

In [39]:
print(len(cpu_ep_data))
condition = cpu_ep_data['label'].str.contains('torchscript')
cpu_ep_data = remove_condition_rows( condition, cpu_ep_data)
print(len(cpu_ep_data))

print("--")
print(len(gpu_ep_data))
condition = gpu_ep_data['label'].str.contains('torchscript')
gpu_ep_data = remove_condition_rows( condition, gpu_ep_data)
print(len(gpu_ep_data))

48
36
--
36
24


In [40]:
print(cpu_ep_data.columns)
print(gpu_ep_data.columns)

assert 'label' in cpu_ep_data.columns
assert 'label' in gpu_ep_data.columns
assert dependent_variable in cpu_ep_data.columns
assert dependent_variable in gpu_ep_data.columns



Index(['runtime', 'model', 'label', 'duration'], dtype='object')
Index(['runtime', 'model', 'label', 'duration'], dtype='object')


In [41]:
if dependent_variable == 'global_energy':
    cpu_ep_data['GPU_energy'] = 0
    cpu_ep_data['global_energy'] = cpu_ep_data['CPU_energy']
    
cpu_ep_data.head()

,runtime,model,label,duration
0,onnx,bloomz-1b1,onnx_bloomz-1b1,1399.077517
1,onnx,bloomz-560m,onnx_bloomz-560m,615.157070
2,onnx,codegemma-2b,onnx_codegemma-2b,3721.251811
3,onnx,codeparrot-small,onnx_codeparrot-small,400.491458
4,onnx,phi2,onnx_phi2,4454.560860


In [42]:
cpu_ep_data['config'] = "cpuep_"+cpu_ep_data['label']
gpu_ep_data['config'] = "gpuep_"+gpu_ep_data['label']

gpu_ep_data

,runtime,model,label,duration,config
0,onnx,bloomz-1b1,onnx_bloomz-1b1,1029.257360,gpuep_onnx_bloomz-1b1
1,onnx,bloomz-560m,onnx_bloomz-560m,225.435832,gpuep_onnx_bloomz-560m
2,onnx,codegemma-2b,onnx_codegemma-2b,678.768990,gpuep_onnx_codegemma-2b
3,onnx,codeparrot-small,onnx_codeparrot-small,191.048335,gpuep_onnx_codeparrot-small
4,onnx,phi2,onnx_phi2,793.478886,gpuep_onnx_phi2
5,onnx,pythia-410m,onnx_pythia-410m,447.527716,gpuep_onnx_pythia-410m
6,onnx,pythia1-4b,onnx_pythia1-4b,535.354030,gpuep_onnx_pythia1-4b
7,onnx,stablecode-3b,onnx_stablecode-3b,617.516664,gpuep_onnx_stablecode-3b
8,onnx,stablecode-3b-completion,onnx_stablecode-3b-completion,964.568877,gpuep_onnx_stablecode-3b-completion
9,onnx,starcoderbase-1b,onnx_starcoderbase-1b,556.800123,gpuep_onnx_starcoderbase-1b


In [43]:
global_df = pd.concat([cpu_ep_data,gpu_ep_data], axis=0, ignore_index=True)
print(global_df['config'].unique())
global_df

['cpuep_onnx_bloomz-1b1' 'cpuep_onnx_bloomz-560m'
 'cpuep_onnx_codegemma-2b' 'cpuep_onnx_codeparrot-small' 'cpuep_onnx_phi2'
 'cpuep_onnx_pythia-410m' 'cpuep_onnx_pythia1-4b'
 'cpuep_onnx_stablecode-3b' 'cpuep_onnx_stablecode-3b-completion'
 'cpuep_onnx_starcoderbase-1b' 'cpuep_onnx_tiny-starcoder'
 'cpuep_onnx_tinyllama' 'cpuep_ov_bloomz-1b1' 'cpuep_ov_bloomz-560m'
 'cpuep_ov_codegemma-2b' 'cpuep_ov_codeparrot-small' 'cpuep_ov_phi2'
 'cpuep_ov_pythia-410m' 'cpuep_ov_pythia1-4b' 'cpuep_ov_stablecode-3b'
 'cpuep_ov_stablecode-3b-completion' 'cpuep_ov_starcoderbase-1b'
 'cpuep_ov_tiny-starcoder' 'cpuep_ov_tinyllama' 'cpuep_torch_bloomz-1b1'
 'cpuep_torch_bloomz-560m' 'cpuep_torch_codegemma-2b'
 'cpuep_torch_codeparrot-small' 'cpuep_torch_phi2'
 'cpuep_torch_pythia-410m' 'cpuep_torch_pythia1-4b'
 'cpuep_torch_stablecode-3b' 'cpuep_torch_stablecode-3b-completion'
 'cpuep_torch_starcoderbase-1b' 'cpuep_torch_tiny-starcoder'
 'cpuep_torch_tinyllama' 'gpuep_onnx_bloomz-1b1' 'gpuep_onnx_bloomz

,runtime,model,label,duration,config
0,onnx,bloomz-1b1,onnx_bloomz-1b1,1399.077517,cpuep_onnx_bloomz-1b1
1,onnx,bloomz-560m,onnx_bloomz-560m,615.157070,cpuep_onnx_bloomz-560m
2,onnx,codegemma-2b,onnx_codegemma-2b,3721.251811,cpuep_onnx_codegemma-2b
3,onnx,codeparrot-small,onnx_codeparrot-small,400.491458,cpuep_onnx_codeparrot-small
4,onnx,phi2,onnx_phi2,4454.560860,cpuep_onnx_phi2
5,onnx,pythia-410m,onnx_pythia-410m,1051.423814,cpuep_onnx_pythia-410m
6,onnx,pythia1-4b,onnx_pythia1-4b,2570.591944,cpuep_onnx_pythia1-4b
7,onnx,stablecode-3b,onnx_stablecode-3b,2408.908307,cpuep_onnx_stablecode-3b
8,onnx,stablecode-3b-completion,onnx_stablecode-3b-completion,5248.014793,cpuep_onnx_stablecode-3b-completion
9,onnx,starcoderbase-1b,onnx_starcoderbase-1b,2432.799547,cpuep_onnx_starcoderbase-1b


In [44]:
print("CPUEP")
print(cpu_ep_data.describe())

print("CUDAEP")
print(gpu_ep_data.describe())


CPUEP
          duration
count    36.000000
mean   2217.442319
std    1784.698451
min     285.900419
25%     658.815203
50%    1672.179387
75%    3084.730893
max    7379.742725
CUDAEP
          duration
count    24.000000
mean    370.791147
std     275.461925
min      94.641196
25%     189.125932
50%     235.414242
75%     540.715553
max    1029.257360


In [45]:
global_df.describe()

,duration
count,60.000000
mean,1478.781850
std,1658.727094
min,94.641196
25%,353.971203
50%,669.721008
75%,2331.817578
max,7379.742725


In [46]:
df = global_df

def get_min_max_config(group):
    min_config = group.loc[group[dependent_variable].idxmin(), 'config']
    max_config = group.loc[group[dependent_variable].idxmax(), 'config']
    return pd.Series({'Min_Config': min_config, 'Max_Config': max_config})

# Applying the function to each group
result = df.groupby('model').apply(get_min_max_config)

print(dependent_variable)
# Output the result
result

duration


,Min_Config,Max_Config
model,,
bloomz-1b1,gpuep_torch_bloomz-1b1,cpuep_onnx_bloomz-1b1
bloomz-560m,gpuep_torch_bloomz-560m,cpuep_torch_bloomz-560m
codegemma-2b,gpuep_torch_codegemma-2b,cpuep_torch_codegemma-2b
codeparrot-small,gpuep_torch_codeparrot-small,cpuep_torch_codeparrot-small
phi2,gpuep_torch_phi2,cpuep_torch_phi2
pythia-410m,gpuep_torch_pythia-410m,cpuep_torch_pythia-410m
pythia1-4b,gpuep_torch_pythia1-4b,cpuep_torch_pythia1-4b
stablecode-3b,gpuep_torch_stablecode-3b,cpuep_torch_stablecode-3b
stablecode-3b-completion,gpuep_torch_stablecode-3b-completion,cpuep_torch_stablecode-3b-completion


In [47]:
df

,runtime,model,label,duration,config
0,onnx,bloomz-1b1,onnx_bloomz-1b1,1399.077517,cpuep_onnx_bloomz-1b1
1,onnx,bloomz-560m,onnx_bloomz-560m,615.157070,cpuep_onnx_bloomz-560m
2,onnx,codegemma-2b,onnx_codegemma-2b,3721.251811,cpuep_onnx_codegemma-2b
3,onnx,codeparrot-small,onnx_codeparrot-small,400.491458,cpuep_onnx_codeparrot-small
4,onnx,phi2,onnx_phi2,4454.560860,cpuep_onnx_phi2
5,onnx,pythia-410m,onnx_pythia-410m,1051.423814,cpuep_onnx_pythia-410m
6,onnx,pythia1-4b,onnx_pythia1-4b,2570.591944,cpuep_onnx_pythia1-4b
7,onnx,stablecode-3b,onnx_stablecode-3b,2408.908307,cpuep_onnx_stablecode-3b
8,onnx,stablecode-3b-completion,onnx_stablecode-3b-completion,5248.014793,cpuep_onnx_stablecode-3b-completion
9,onnx,starcoderbase-1b,onnx_starcoderbase-1b,2432.799547,cpuep_onnx_starcoderbase-1b


In [48]:
def get_min_max_config(group):
    # Find the row with the minimum value of the dependent variable
    min_row = group.loc[group[dependent_variable].idxmin()]
    # Find the row with the maximum value of the dependent variable
    max_row = group.loc[group[dependent_variable].idxmax()]
    
    # Return both config and values
    return pd.Series({
        'Min_Config': min_row['config'],
        'Min_Value': min_row[dependent_variable],
        'Max_Config': max_row['config'],
        'Max_Value': max_row[dependent_variable],
        'x_times': max_row[dependent_variable]/min_row[dependent_variable],
        'percentage':min_row[dependent_variable]/max_row[dependent_variable]
    })

# Applying the function to each group
result = df.groupby('model').apply(get_min_max_config)

# Output the result
result

,Min_Config,Min_Value,Max_Config,Max_Value,x_times,percentage
model,,,,,,
bloomz-1b1,gpuep_torch_bloomz-1b1,105.560827,cpuep_onnx_bloomz-1b1,1399.077517,13.253757,0.075450
bloomz-560m,gpuep_torch_bloomz-560m,117.582811,cpuep_torch_bloomz-560m,660.673025,5.618789,0.177974
codegemma-2b,gpuep_torch_codegemma-2b,226.349586,cpuep_torch_codegemma-2b,4951.058533,21.873504,0.045717
codeparrot-small,gpuep_torch_codeparrot-small,94.684550,cpuep_torch_codeparrot-small,472.177037,4.986844,0.200528
phi2,gpuep_torch_phi2,251.969879,cpuep_torch_phi2,5983.522681,23.746976,0.042111
pythia-410m,gpuep_torch_pythia-410m,189.228687,cpuep_torch_pythia-410m,1306.998445,6.906978,0.144781
pythia1-4b,gpuep_torch_pythia1-4b,190.243684,cpuep_torch_pythia1-4b,3606.597118,18.957776,0.052749
stablecode-3b,gpuep_torch_stablecode-3b,187.495280,cpuep_torch_stablecode-3b,4361.323101,23.260975,0.042990
stablecode-3b-completion,gpuep_torch_stablecode-3b-completion,376.661465,cpuep_torch_stablecode-3b-completion,7379.742725,19.592508,0.051040


In [49]:
subset_cpu = global_df[global_df['config'].str.startswith('cpu', na=False)]
subset_gpu = global_df[global_df['config'].str.startswith('gpu', na=False)]

In [50]:
# conclusions
# Applying the function to each group
percentages_cpu = subset_cpu.groupby('model').apply(get_min_max_config)

print(f"DV: {dependent_variable}")
print(f"Savings from {percentages_cpu['percentage'].min()} to {percentages_cpu['percentage'].max()}")

# Output the result
percentages_cpu

DV: duration
Savings from 0.2879082446820076 to 0.7903510589906054


,Min_Config,Min_Value,Max_Config,Max_Value,x_times,percentage
model,,,,,,
bloomz-1b1,cpuep_ov_bloomz-1b1,575.626701,cpuep_onnx_bloomz-1b1,1399.077517,2.430529,0.411433
bloomz-560m,cpuep_ov_bloomz-560m,455.812262,cpuep_torch_bloomz-560m,660.673025,1.449441,0.689921
codegemma-2b,cpuep_ov_codegemma-2b,2311.134276,cpuep_torch_codegemma-2b,4951.058533,2.142263,0.466796
codeparrot-small,cpuep_ov_codeparrot-small,285.900419,cpuep_torch_codeparrot-small,472.177037,1.651544,0.605494
phi2,cpuep_ov_phi2,2393.867481,cpuep_torch_phi2,5983.522681,2.499521,0.400077
pythia-410m,cpuep_ov_pythia-410m,1032.987605,cpuep_torch_pythia-410m,1306.998445,1.265261,0.790351
pythia1-4b,cpuep_ov_pythia1-4b,1705.546433,cpuep_torch_pythia1-4b,3606.597118,2.114629,0.472896
stablecode-3b,cpuep_ov_stablecode-3b,1255.660879,cpuep_torch_stablecode-3b,4361.323101,3.473329,0.287908
stablecode-3b-completion,cpuep_ov_stablecode-3b-completion,3239.888109,cpuep_torch_stablecode-3b-completion,7379.742725,2.277777,0.439025


In [51]:
# Applying the function to each group
percentages_gpu = subset_gpu.groupby('model').apply(get_min_max_config)

print(f"DV: {dependent_variable}")

print(f"Savings from {percentages_gpu['percentage'].min()} to {percentages_gpu['percentage'].max()}")
# Output the result
percentages_gpu


DV: duration
Savings from 0.10256018669616314 to 0.5215799556691146


,Min_Config,Min_Value,Max_Config,Max_Value,x_times,percentage
model,,,,,,
bloomz-1b1,gpuep_torch_bloomz-1b1,105.560827,gpuep_onnx_bloomz-1b1,1029.257360,9.750372,0.102560
bloomz-560m,gpuep_torch_bloomz-560m,117.582811,gpuep_onnx_bloomz-560m,225.435832,1.917252,0.521580
codegemma-2b,gpuep_torch_codegemma-2b,226.349586,gpuep_onnx_codegemma-2b,678.768990,2.998764,0.333471
codeparrot-small,gpuep_torch_codeparrot-small,94.684550,gpuep_onnx_codeparrot-small,191.048335,2.017735,0.495605
phi2,gpuep_torch_phi2,251.969879,gpuep_onnx_phi2,793.478886,3.149102,0.317551
pythia-410m,gpuep_torch_pythia-410m,189.228687,gpuep_onnx_pythia-410m,447.527716,2.365010,0.422831
pythia1-4b,gpuep_torch_pythia1-4b,190.243684,gpuep_onnx_pythia1-4b,535.354030,2.814044,0.355361
stablecode-3b,gpuep_torch_stablecode-3b,187.495280,gpuep_onnx_stablecode-3b,617.516664,3.293505,0.303628
stablecode-3b-completion,gpuep_torch_stablecode-3b-completion,376.661465,gpuep_onnx_stablecode-3b-completion,964.568877,2.560838,0.390497


In [18]:
#result = subset_cpu.groupby('model').apply(get_min_max_config)
#result

In [19]:
#result = subset_gpu.groupby('model').apply(get_min_max_config)
#result

In [20]:
# Sort DataFrame by 'model' and 'dependent_variable' descending
sorted_df = subset_cpu.sort_values(by=['model',dependent_variable], ascending=[True, False])

# Group by model and select runtime_engines in order of descending dependent_variable
result = sorted_df.groupby('model')['runtime'].apply(list).reset_index()

# Display the result
print(f"Order {dependent_variable} by model: CPUExecutionProvider")
result

Order global_energy by model: CPUExecutionProvider


,model,runtime
0,bloomz-1b1,"[onnx, ov, torch]"
1,bloomz-560m,"[torch, ov, onnx]"
2,codegemma-2b,"[ov, torch, onnx]"
3,codeparrot-small,"[torch, onnx, ov]"
4,idle,[idle]
5,phi2,"[torch, onnx, ov]"
6,pythia-410m,"[ov, torch, onnx]"
7,pythia1-4b,"[torch, ov, onnx]"
8,stablecode-3b,"[torch, onnx, ov]"
9,stablecode-3b-completion,"[torch, ov, onnx]"


In [21]:
# Sort DataFrame by 'model' and 'dependent_variable' descending
sorted_df = subset_gpu.sort_values(by=['model', dependent_variable], ascending=[True, False])

# Group by model and select runtime_engines in order of descending dependent_variable
result = sorted_df.groupby('model')['runtime'].apply(list).reset_index()

# Display the result
print("Order by model")
print(f"Order {dependent_variable} by model: GPUExecutionProvider")
result

Order by model
Order global_energy by model: GPUExecutionProvider


,model,runtime
0,bloomz-1b1,"[onnx, torch]"
1,bloomz-560m,"[onnx, torch]"
2,codegemma-2b,"[onnx, torch]"
3,codeparrot-small,"[onnx, torch]"
4,idle,[idle]
5,phi2,"[onnx, torch]"
6,pythia-410m,"[onnx, torch]"
7,pythia1-4b,"[onnx, torch]"
8,stablecode-3b,"[onnx, torch]"
9,stablecode-3b-completion,"[onnx, torch]"


In [22]:
# all ordered by dependent variable

global_df.sort_values(by=dependent_variable, ascending=False)

,runtime,model,label,experiment,CPU_energy,GPU_energy,global_energy,config
33,torch,stablecode-3b-completion,torch_stablecode-3b-completion,5.5,631788.437874,0.000000,631788.437874,cpuep_torch_stablecode-3b-completion
32,torch,stablecode-3b,torch_stablecode-3b,5.5,589430.745087,0.000000,589430.745087,cpuep_torch_stablecode-3b
21,ov,stablecode-3b-completion,ov_stablecode-3b-completion,5.5,585458.906937,0.000000,585458.906937,cpuep_ov_stablecode-3b-completion
9,onnx,stablecode-3b-completion,onnx_stablecode-3b-completion,5.5,575477.807861,0.000000,575477.807861,cpuep_onnx_stablecode-3b-completion
29,torch,phi2,torch_phi2,5.5,494766.105080,0.000000,494766.105080,cpuep_torch_phi2
...,...,...,...,...,...,...,...,...
60,torch,tiny-starcoder,torch_tiny-starcoder,5.5,4246.124115,8738.370629,12977.583165,gpuep_torch_tiny-starcoder
53,torch,codeparrot-small,torch_codeparrot-small,5.5,4424.862007,8426.396317,12825.179010,gpuep_torch_codeparrot-small
50,torch,bloomz-1b1,torch_bloomz-1b1,5.5,2628.826508,9751.112565,12388.138642,gpuep_torch_bloomz-1b1
0,idle,idle,idle_idle,5.5,5450.420303,0.000000,5450.420303,cpuep_idle_idle


In [23]:
def remove_condition_rows(condition_to_remove,df):
    # example: condition = merged_df['label']  == 'idle_idle'
    df1 = df.copy()
    return df1[~condition_to_remove]

In [24]:
condition = global_df['config'].str.endswith('idle')
global_df = remove_condition_rows( condition, global_df)
global_df.sort_values(by=dependent_variable, ascending=False)

,runtime,model,label,experiment,CPU_energy,GPU_energy,global_energy,config
33,torch,stablecode-3b-completion,torch_stablecode-3b-completion,5.5,631788.437874,0.000000,631788.437874,cpuep_torch_stablecode-3b-completion
32,torch,stablecode-3b,torch_stablecode-3b,5.5,589430.745087,0.000000,589430.745087,cpuep_torch_stablecode-3b
21,ov,stablecode-3b-completion,ov_stablecode-3b-completion,5.5,585458.906937,0.000000,585458.906937,cpuep_ov_stablecode-3b-completion
9,onnx,stablecode-3b-completion,onnx_stablecode-3b-completion,5.5,575477.807861,0.000000,575477.807861,cpuep_onnx_stablecode-3b-completion
29,torch,phi2,torch_phi2,5.5,494766.105080,0.000000,494766.105080,cpuep_torch_phi2
5,onnx,phi2,onnx_phi2,5.5,475452.949699,0.000000,475452.949699,cpuep_onnx_phi2
17,ov,phi2,ov_phi2,5.5,442486.860435,0.000000,442486.860435,cpuep_ov_phi2
15,ov,codegemma-2b,ov_codegemma-2b,5.5,409354.084854,0.000000,409354.084854,cpuep_ov_codegemma-2b
27,torch,codegemma-2b,torch_codegemma-2b,5.5,334939.674240,0.000000,334939.674240,cpuep_torch_codegemma-2b
31,torch,pythia1-4b,torch_pythia1-4b,5.5,322246.035751,0.000000,322246.035751,cpuep_torch_pythia1-4b
